In [2]:
import pandas as pd
from flask import (
    Flask,
    json,
    render_template,
    jsonify)

In [3]:
#for the /names app
sample = pd.read_csv("belly_button_biodiversity_samples.csv")
otu_id = pd.read_csv("belly_button_biodiversity_otu_id.csv")
metadata = pd.read_csv("Belly_Button_Biodiversity_Metadata.csv")
otu_in_samples = pd.read_csv("belly_button_biodiversity_samples.csv")

In [4]:
sample.head()

,otu_id,BB_940,BB_941,BB_943,BB_944,BB_945,BB_946,BB_947,BB_948,BB_949,...,BB_1562,BB_1563,BB_1564,BB_1572,BB_1573,BB_1574,BB_1576,BB_1577,BB_1581,BB_1601
0,1,0.0,0,0,0,0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0.0,0,0,0,0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0.0,0,0,0,0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0.0,0,0,0,0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0.0,0,0,0,0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
#list of sample names
sample_names = list(sample.columns[1:])

In [6]:
#for @app.route('/otu')
otu_id.head()

,otu_id,lowest_taxonomic_unit_found
0,1,Archaea;Euryarchaeota;Halobacteria;Halobacteri...
1,2,Archaea;Euryarchaeota;Halobacteria;Halobacteri...
2,3,Archaea;Euryarchaeota;Halobacteria;Halobacteri...
3,4,Archaea;Euryarchaeota;Methanobacteria;Methanob...
4,5,Archaea;Euryarchaeota;Methanobacteria;Methanob...


In [7]:
#list of OTU descriptions
otu_descr = list(otu_id["lowest_taxonomic_unit_found"])

In [8]:
#for the @app.route('/metadata/<sample>') & @app.route('/wfreq/<sample>')
metadata.head()

,SAMPLEID,EVENT,ETHNICITY,GENDER,AGE,WFREQ,BBTYPE,LOCATION,COUNTRY012,ZIP012,...,DOG,CAT,IMPSURFACE013,NPP013,MMAXTEMP013,PFC013,IMPSURFACE1319,NPP1319,MMAXTEMP1319,PFC1319
0,940,BellyButtonsScienceOnline,Caucasian,F,24.0,2.0,I,Beaufort/NC,usa,22306,...,no,no,8852.0,37.172222,54.5,NaN,1,NaN,33.990002,25.5
1,941,NaN,Caucasian/Midleastern,F,34.0,1.0,I,Chicago/IL,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,943,BellyButtonsScienceOnline,Caucasian,F,49.0,1.0,I,Omaha/NE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,944,BellyButtonsScienceOnline,European,M,44.0,1.0,I,NewHaven/CT,usa,7079,...,no,yes,NaN,35.816666,16.0,NaN,0,6567.0,32.403332,28.5
4,945,BellyButtonsScienceOnline,Caucasian,F,48.0,1.0,I,Philidelphia/PA,usa,84404,...,no,no,NaN,37.783333,4.0,NaN,0,5613.0,33.634445,24.0


In [9]:
new_metadata = metadata[["AGE", "BBTYPE", "ETHNICITY", "GENDER", "LOCATION", "SAMPLEID", "WFREQ"]]

In [10]:
meta_dict = list(new_metadata.T.to_dict().values())
type(meta_dict)

list

In [11]:
#meta_dict is a list of dictionaries
meta_dict[0]

{'AGE': 24.0,
 'BBTYPE': 'I',
 'ETHNICITY': 'Caucasian',
 'GENDER': 'F',
 'LOCATION': 'Beaufort/NC',
 'SAMPLEID': 940,
 'WFREQ': 2.0}

In [12]:
# for @app.route('/samples/<sample>')
otu_in_samples.head()

,otu_id,BB_940,BB_941,BB_943,BB_944,BB_945,BB_946,BB_947,BB_948,BB_949,...,BB_1562,BB_1563,BB_1564,BB_1572,BB_1573,BB_1574,BB_1576,BB_1577,BB_1581,BB_1601
0,1,0.0,0,0,0,0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0.0,0,0,0,0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0.0,0,0,0,0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0.0,0,0,0,0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0.0,0,0,0,0,0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
samples_otu_lst = []
for col in otu_in_samples.columns[1:]:

    sample_pd = pd.DataFrame(columns=["otu_id","num_samples"])
    otu_id_lst = []
    sample_val_lst = []
    
    non_zero_index = [x for x in otu_in_samples[col].index if otu_in_samples[col][x] > 0]
    
    for indx in non_zero_index:
        otu_id_lst.append(otu_in_samples["otu_id"][indx])
        sample_val_lst.append(otu_in_samples[col][indx])
        
    sample_pd["otu_id"] = otu_id_lst
    sample_pd["num_samples"] = sample_val_lst

    sample_pd = sample_pd.sort_values('num_samples',ascending=False)
    
    sample = {
#         "sample_id" : col,
        "otu_id": str(list(sample_pd["otu_id"])),
        "sample_values": str(list(sample_pd["num_samples"]))
    }
    
    samples_otu_lst.append(sample)
    

In [30]:
sample_pd.reset_index()
sample_pd.head()

,otu_id,num_samples
66,1929,166
97,3030,129
45,1795,97
13,894,94
23,1167,73


In [ ]:
#list of dictionaries with otu_id & sample values for every sample_id
for x in samples_otu_lst[1:3]:
    print(x["sample_id"].split("_"))